In [1]:
import requests
import pandas as pd

from tqdm import tqdm

In [8]:
def get_city(tag: str) -> dict:
    
    """
    Acessa as informações de uma cidade pela API do inteli.gente
    
    Parameters
    ----------
    tag: str
        A tag da cidade para acessar o end-point na API.
    
    Returns
    -------
    dict
        Dicionário com os dados que a API retornou sobre a cidade.
    """
    
    
    # Headers para simular um navegador real
    headers = {
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
        'origin': 'https://inteligente.mcti.gov.br',
        'priority': 'u=1, i',
        'sec-ch-ua': '"Chromium";v="124", "Google Chrome";v="124", "Not-A.Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Linux"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
    }

    # Realiza a requisição ao end-point na API correspondente a cidade que queremos acessar os dados
    response = requests.get(f'https://inteligente-backend.mcti.gov.br/api/cities/{tag}', headers=headers)
    
    # Retorna um dicionário com os dados da API
    return response.json()

In [3]:
def get_city_codes():

    """
    Função para listar todas as cidades disponíveis na API do inteli.gente
    """    
    

    # Headers para simular um navegador real
    headers = {
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
        'origin': 'https://inteligente.mcti.gov.br',
        'priority': 'u=1, i',
        'sec-ch-ua': '"Chromium";v="124", "Google Chrome";v="124", "Not-A.Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Linux"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
    }

    response = []

    for letter in tqdm(list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")):

        # Pesquisa cada letra do alfabeto na barra de pesquisa e salva todos os resultados.
        
        params = {
            'term': letter,
        }

        response += requests.get('https://inteligente-backend.mcti.gov.br/api/cities/autocomplete', params=params, headers=headers).json()
        
    return pd.DataFrame(response)

In [4]:
codes = get_city_codes()
codes

100%|███████████████████████████████████████████| 26/26 [00:13<00:00,  2.00it/s]


,id,friendlyName,result
0,5200050,abadia-de-goias,"Abadia de Goiás, GO"
1,3100104,abadia-dos-dourados,"Abadia dos Dourados, MG"
2,5200100,abadiania,"Abadiânia, GO"
3,3100203,abaete,"Abaeté, MG"
4,1500107,abaetetuba,"Abaetetuba, PA"
...,...,...,...
5565,2933604,xique-xique,"Xique-Xique, BA"
5566,2517407,zabele,"Zabelê, PB"
5567,3557154,zacarias,"Zacarias, SP"
5568,2114007,ze-doca,"Zé Doca, MA"


In [ ]:
features = []

for code in tqdm(codes['friendlyName'].tolist()):
    
    # Acessa os dados da cidade
    r = get_city(code)

    # Tabela com dados da Caracterização sociodemográfica
    C = pd.DataFrame(r['characterization'])

    # Dicionario com níveis de cada dimensão
    levels = {dim['code']: dim['level'] for dim in r['dimensions']}

    metadata = {
        "codigo": r['code'],
        "tag": r['friendlyName'],
        "nome": f"{r['name']} ({r['stateAbbr']})",
        "latitude": r['latitude'],
        "longitude": r['longitude'],
        "Nivel Sociocultural": levels['d4'],
        "Nivel Economica": levels['d2'],
        "Nivel Capacidades Institucionais": levels['d6'],
        "Nivel Meio Ambiente": levels['d3'],

        "PIB per capita do municipio": C[C['code'] == 'i3087']['value'].item(),
        "PIB Serviços": C[C['code'] == 'i4058']['value'].item(),
        "PIB Industria": C[C['code'] == 'i4057']['value'].item(),
        "PIB Adminstração Publica": C[C['code'] == 'i4059']['value'].item(),
        "PIB Agropecuária": C[C['code'] == 'i4056']['value'].item(),
        
        "Populacao Estimada": C[C['code'] == 'i4003']['value'].item(),
        "População ocupada com vinculo formal": C[C['code'] == 'i3135']['value'].item(),
    }
    
    features.append(metadata)
    
pd.DataFrame(features).to_csv("results_PCDII.csv")

In [20]:
# Celula para baixar labels da API

headers = {
    'sec-ch-ua': '"Chromium";v="124", "Google Chrome";v="124", "Not-A.Brand";v="99"',
    'Accept': 'application/json, text/plain, */*',
    'Referer': '',
    'sec-ch-ua-mobile': '?0',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
    'sec-ch-ua-platform': '"Linux"',
}

response = requests.get('https://inteligente-backend.mcti.gov.br/api/cities/score/labels', headers=headers)

labels = pd.DataFrame(response.json()).T[['title', 'description', 'valueUnit']]
labels.to_csv("labels.csv")